In [2]:
#Fetch Gemini API Key
import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("Gemini API Key:")

**bbc** - 크롤링 불가, 유기 <br>
**블랙월** - 광고 너무 많아서 유기, 요약문만 사용? <br>
**뉴욕타임즈** - API사용, 요약본, URL, 타이틀<br>
**CNN** - 전문 긁어오기

In [48]:
import numpy
import pandas as pd

In [ ]:
# cnn 기사 데이터 수집
# cnn_news.csv 파일 경로 수정
cnn=pd.read_csv("../data/cnn_news.csv")

# 기사 선정 기준? 
import random
while True: 
    i=int(random.random()*100)
    article=cnn.iloc[i]
    if not (article.content=='' or article.content=="No content available."): break
article

title      Italy releases Iranian businessman wanted by U...
link       https://edition.cnn.com/2025/01/12/europe/ital...
content    Iranian businessman Mohammad Abedini, who was ...
Name: 4, dtype: object

In [88]:
print(article.content)

Iranian businessman Mohammad Abedini, who was detained in Italy at the request of the United States following a drone attack that killed three Americans, has been released. Abedini, 38, was arrested i...


# article
A news summary from Apple falsely claimed darts player Luke Littler had won the PDC World Championship - before he even played in the final.
The incorrect summary was written by artificial intelligence (AI) and is based on a BBC story about Littler winning the tournament semi-final on Thursday night.
Within hours on Friday, another AI notification summary falsely told some BBC Sport app users that Tennis great Rafael Nadal had come out as gay.
Apple, which has been contacted for a response, previously declined to comment on similar examples of false AI alerts about news stories.
Apple's AI software Apple Intelligence launched in the UK in December 2024 with a feature that aims to give users a simple round-up of app alerts they have missed.
It collates the alerts into a single message and then uses AI to summarise what they contained.
The false alert about Littler was seen by users of the BBC News app earlier in the day.
Meanwhile, a separate AI summary told some users of the BBC Sport app that "Brazilian tennis player, Rafael Nadal, comes out as gay".
That false alert appeared to relate to a story about "trailblazing" Brazilian gay tennis player Joao Lucas Reis da Silva and the impact of his openness about his sexuality on the sport more widely.
A BBC spokesperson said: "It is essential that Apple fixes this problem urgently - as this has happened multiple times.
"As the most trusted news media organisation in the world, it is crucial that audiences can trust any information or journalism published in our name and that includes notifications."
The BBC previously complained to Apple about the Apple Intelligence feature when it generated a false headline about a high-profile alleged murder in the US.
Even though this type of summary notification appears to be directly from the BBC, it is in fact Apple Intelligence's take on much longer headlines.
Other stories in the summaries seen on Friday - including those on South Korea and rising cases of influenza - were reported accurately by Apple Intelligence.
The latest examples come after journalist body Reporters Without Borders (also known as RSF) called on Apple to axe its AI-powered summaries of news stories last month.
"The automated production of false information attributed to a media outlet is a blow to the outlet's credibility," Vincent Berthier, the head of RSF's technology and journalism desk, said in December.
He added it was "a danger to the public's right to reliable information on current affairs."
It is likely that most people's AI-generated summaries are unique as different combinations of notifications are summarised depending on the device being used and the alerts coming through.
BBC Sport app users can follow different sports and receive personalised alerts.
Apple Intelligence is only available on certain iPhones - those using the iOS 18.1 system version or later on recent devices (all iPhone 16 phones, the 15 Pro, and the 15 Pro Max). It is also available on some iPads and Macs.
The grouped notifications are marked with a specific icon, and users can report any concerns they have on a notification summary.
Apple has not outlined how many reports it has received.

In [94]:
# CNN full code
import time
import google.generativeai as genai


# model for CNN, summarize
# flash와 pro 양자택일, 어느쪽을 택하든 코드가 크게 달라지진 않음
#system_instruction=
model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-pro")

# give model an article
chat = model.start_chat(history=[{'role':'user', 'parts':article.content}])

# create summary
start=time.time()
summary = article.title
summary += chat.send_message("입력된 내용을 영어로 요약해서 출력해줘. 요약문은 100자 이내여야 해.").text
summary += chat.send_message("요약본을 한국어로 번역해줘").text
end=time.time()
print(f"summary created: {end - start:.5f}sec")

# create vocab
start=time.time()
prompt="""
영문 요약본에서 적당한 난이도의 단어 3개만 찾아줘, 단어 옆에  한국어 뜻을 붙여줘. 단어와 뜻만 출력해. 각 단어 사이에는 개행해. 
예시: 
collates 모으다
Crucial 중요한
Attributed 귀속된
"""

response = chat.send_message(prompt)
response=response.text.split('\n')
response.pop()
vocab={}
for i in range(len(response)):
    v,m=response[i].split(maxsplit=1)
    vocab[v]=m
end=time.time()
print(f"vocab created: {end - start:.5f}sec")

summary created: 2.72702sec
vocab created: 0.49353sec


In [ ]:
#summary 생성 셀
# flash와 pro 양자택일, 어느쪽을 택하든 코드가 크게 달라지진 않음
#system_instruction=
model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-pro")

# give model an article
chat = model.start_chat(history=[{'role':'user', 'parts':article}])

# create summary
start=time.time()
summary = chat.send_message("가지고 온 영어로된 기사를 영어로 요약해서 출력해줘. 요약문은 100자 이내여야 해.")
summary += chat.send_message("요약본을 한국어로 번역해줘")
end=time.time()
print(f"summary created: {end - start:.5f}sec")

In [95]:
print(summary)

Italy releases Iranian businessman wanted by US over drone attack that killed AmericansIranian businessman Mohammad Abedini, 38, was detained in Italy at the US's request following a drone strike that killed three Americans.  He has since been released.  The reason for his detention and the specifics of the drone strike remain unclear.
38세 이란 사업가 모하마드 아베디니가 미국인 3명이 사망한 드론 공격 이후 미국의 요청으로 이탈리아에서 체포되었으나 석방되었습니다.  체포 이유와 드론 공격의 구체적인 내용은 아직 불분명합니다.



 # Summary
Apple's AI falsely reported that Luke Littler won the PDC World Championship before the final and that Rafael Nadal came out as gay, sparking concerns about the credibility of AI-generated news summaries.

In [89]:
print("원문 길이:", len(article.content))
print("요약문 길이:", len(summary.split("\n")[0]))

원문 길이: 203
요약문 길이: 339


In [97]:
# 영어 요약문
summary.split("\n")[0]

# 한국어 요약문
summary.split("\n")[1]

'38세 이란 사업가 모하마드 아베디니가 미국인 3명이 사망한 드론 공격 이후 미국의 요청으로 이탈리아에서 체포되었으나 석방되었습니다.  체포 이유와 드론 공격의 구체적인 내용은 아직 불분명합니다.'

In [45]:
# 단어 추출
# flash와 pro 양자택일, 어느쪽을 택하든 코드가 크게 달라지진 않음
model = genai.GenerativeModel("gemini-1.5-flash")
#model = genai.GenerativeModel("gemini-pro")

# give model an article
chat = model.start_chat(history=[{'role':'user', 'parts':article}])

# create vocab
start=time.time()
prompt="""
영문 요약본에서 적당한 난이도의 단어 3개만 찾아줘, 단어 옆에  한국어 뜻을 붙여줘. 단어와 뜻만 출력해. 각 단어 사이에는 개행해. 
예시: 
collates 모으다
Crucial 중요한
Attributed 귀속된
"""

response = chat.send_message(prompt)
response=response.text.split('\n')
response.pop()
vocab={}
for i in range(len(response)):
    v,m=response[i].split(maxsplit=1)
    vocab[v]=m
end=time.time()
print(f"vocab created: {end - start:.5f}sec")

vocab created: 0.54811sec


In [91]:
vocab

{'detained': '체포된', 'request': '요청', 'following': '~이후에'}

In [69]:
chat.history

[parts {
   text: "A former child star from Australia died when the Los Angeles wildfires ripped through his family’s Malibu estate in California earlier this week, according to his mother. Rory Callum Sykes was at the ..."
 }
 role: "user",
 parts {
   text: "가지고 온 영어로된 기사를 영어로 요약해서 출력해줘. 요약문은 100자 이내여야 해."
 }
 role: "user",
 parts {
   text: "Please provide me with the English article you\'d like me to summarize.  I need the text of the article to create a 100-word summary.\n"
 }
 role: "model",
 parts {
   text: "요약본을 한국어로 번역해줘"
 }
 role: "user",
 parts {
   text: "죄송합니다. 제게 기사 내용이 없어서 요약본을 한국어로 번역할 수 없습니다. 기사를 제공해주시면 제가 요약하고 한국어로 번역해 드리겠습니다.\n"
 }
 role: "model",
 parts {
   text: "\n영문 요약본에서 적당한 난이도의 단어 3개만 찾아줘, 단어 옆에  한국어 뜻을 붙여줘. 단어와 뜻만 출력해.\n예시: \ncollates 모으다\nCrucial 중요한\nAttributed 귀속된\n"
 }
 role: "user",
 parts {
   text: "estate 재산\nwildfires 산불\nreportedly 보도에 따르면\n"
 }
 role: "model"]